In [1]:
#finding the best validation accuracy
#without handcoded features
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from torch.utils.data import Subset
import wandb
from PIL import Image
from tabulate import tabulate
import numpy as np
import random
import shutil
import matplotlib.pyplot as plt

In [2]:
# Adapter so FineTuneCNN still gets (img, tab, label)
class ImageOnlyDataset(Dataset):
    def __init__(self, ds):
        self.ds = ds
    def __len__(self):
        return len(self.ds)
    def __getitem__(self, idx):
        img, lbl = self.ds[idx]
        tab = torch.zeros(0, dtype=torch.float32)
        return img, tab, lbl


In [3]:
IMG_SIZE = (224, 224)
transform_pipeline = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# Paths
image_root      = '/kaggle/input/train-valid/train'  # train+val
test_image_root = '/kaggle/input/test-data'          # fixed test

random.seed(42)

# Build the full train+val dataset
full_trainval = datasets.ImageFolder(root=image_root, transform=transform_pipeline)
total_tv      = len(full_trainval)
# 70% train, 10% val (total train data is 80% of original)
n_val   = int(0.1 * total_tv)
n_train = total_tv - n_val

train_ds, val_ds = random_split(
    full_trainval,
    [n_train, n_val],
    generator=torch.Generator().manual_seed(42)
)

# 3.3 Load the fixed test dataset (no split)
test_ds = datasets.ImageFolder(
    root=test_image_root,
    transform=transform_pipeline
)

In [4]:
#print the details of dataset
total   = total_tv
n_train = len(train_ds)
n_val   = len(val_ds)
n_test  = len(test_ds)

# class counts in the ORIGINAL image_root
class_counts = []
for cls in sorted(os.listdir(image_root)):
    cls_dir = os.path.join(image_root, cls)
    if not os.path.isdir(cls_dir):
        continue
    cnt = len([
        f for f in os.listdir(cls_dir)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ])
    class_counts.append((cls, cnt))

print(tabulate(class_counts, headers=['Class Name', '# Images']))
print(f"\nTotal images in the train datafolder:       {total}")
print(f"  Train images:                              {n_train}")
print(f"  Val images:                                {n_val}")
print(f"  Test images:                               {n_test}\n")

Class Name                                                   # Images
---------------------------------------------------------  ----------
1. Eczema                                                        1342
10. Warts Molluscum and other Viral Infections                   1683
2. Melanoma                                                      2512
3. Atopic Dermatitis                                             1006
4. Basal Cell Carcinoma                                          2659
5. Melanocytic Nevi                                              6376
6. Benign Keratosis-like Lesions                                 1664
7. Psoriasis pictures Lichen Planus and related diseases         1644
8. Seborrheic Keratoses and other Benign Tumors                  1478
9. Tinea Ringworm Candidiasis and other Fungal Infections        1362

Total images in the train datafolder:       21726
  Train images:                              19554
  Val images:                                2172
  Test i

In [5]:
# EXTEND ResNet TO CONCAT TABULAR FEATURES 
class ResNetWithTabular(nn.Module):
    def __init__(self, base_model, tab_dim, num_classes=10):
        super().__init__()
        # keep everything except the final fully connected layer
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])
        # only apply batch normalization if we actually have tab features
        self.tab_bn   = nn.Identity() if tab_dim == 0 else nn.BatchNorm1d(tab_dim)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(base_model.fc.in_features + tab_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    def forward(self, img, tab):
        x = self.backbone(img).flatten(1)  
        tab = self.tab_bn(tab)             # Identity for tab_dim=0, batch norm when tab_dim=0
        x = torch.cat([x, tab], dim=1)     # works even if tab has shape [B,0]
        return self.classifier(x)



In [6]:
#  FINE‑TUNING CLASS 
class FineTuneCNN:
    def __init__(self, train_ds, valid_ds, base_model, batch_size=32, freeze_ratio=1.0, test_ds=None):
        self.model = base_model
        # freeze parameters
        if freeze_ratio >= 1.0:
            for p in self.model.parameters():
                p.requires_grad = False
        else:
            total_p = sum(1 for _ in self.model.parameters())
            to_freeze = int(total_p * freeze_ratio)
            cnt = 0
            for p in self.model.parameters():
                p.requires_grad = False
                cnt += 1
                if cnt >= to_freeze:
                    break
        # ensure final layers are trainable
        for p in self.model.classifier.parameters():
            p.requires_grad = True

        # override the default loaders
        self.train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        self.valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)
        self.test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False) if test_ds else None

    def run_training(self, num_epochs=10, learning_rate=1e-3, weight_decay_val=0):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(),
                               lr=learning_rate,
                               weight_decay=weight_decay_val)

        for epoch in range(1, num_epochs+1):
            # train 
            self.model.train()
            running_correct = 0
            running_total   = 0
            for img, tab, lbl in self.train_loader:
                img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                optimizer.zero_grad()
                out = self.model(img, tab)
                loss = criterion(out, lbl)
                loss.backward()
                optimizer.step()
                pred = out.argmax(dim=1)
                running_correct += (pred == lbl).sum().item()
                running_total   += lbl.size(0)
            train_acc = 100 * running_correct / running_total
            print(f"Epoch {epoch} — Train Acc: {train_acc:.2f}%")
            wandb.log({"epoch":epoch, "train_acc":train_acc})

            # validate 
            self.model.eval()
            val_corr = 0
            val_tot  = 0
            val_loss = 0.0
            with torch.no_grad():
                for img, tab, lbl in self.valid_loader:
                    img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                    out = self.model(img, tab)
                    l  = criterion(out, lbl)
                    pred = out.argmax(dim=1)
                    val_corr += (pred == lbl).sum().item()
                    val_tot  += lbl.size(0)
                    val_loss  = l.item()
            val_acc = 100 * val_corr / val_tot
            print(f"Epoch {epoch} — Val   Acc: {val_acc:.2f}%")
            wandb.log({"validation_accuracy": val_acc, "validation_loss": val_loss})

    def evaluate_test(self):
        if self.test_loader is None:
            print("No test set.")
            return
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        test_corr = 0
        test_tot  = 0
        with torch.no_grad():
            for img, tab, lbl in self.test_loader:
                img, tab, lbl = img.to(device), tab.to(device), lbl.to(device)
                out = self.model(img, tab)
                pred = out.argmax(dim=1)
                test_corr += (pred == lbl).sum().item()
                test_tot  += lbl.size(0)
        test_acc = 100 * test_corr / test_tot
        print(f"Test Accuracy: {test_acc:.2f}%")
        wandb.log({"test_accuracy": test_acc})



In [7]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [8]:
wandb.login(key='1df7a902fa4a610500b8e79e21818419d5facdbb')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m018 (ma23m018-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
# ---------- 7. MAIN / WANDB SWEEP SETUP ----------
#wandb.login(key='1df7a902fa4a610500b8e79e21818b')

sweep_config = {
    'method': 'bayes',
    'metric': {'goal': 'maximize','name': 'validation_accuracy'},
    'parameters': {
        'learning_rate': {'values':[1e-3,1e-4]},
        'freeze_ratio':  {'values':[0.2,0.6,0.9]},
        'l2_reg':        {'values':[0,5e-4,5e-2]},
        'batch_size':    {'values':[32,64]},
        'epochs':        {'values':[5,10]}
    }
}


In [10]:
sweep_id = wandb.sweep(sweep_config, entity= "ma23m018-indian-institute-of-technology-madras", project="mtech_project_wh1")

Create sweep with ID: 3tql99hs
Sweep URL: https://wandb.ai/ma23m018-indian-institute-of-technology-madras/mtech_project_wh1/sweeps/3tql99hs


In [11]:
def main():
    with wandb.init() as run:
        cfg = wandb.config
        run.name = f"bs{cfg.batch_size}_ep{cfg.epochs}_lr{cfg.learning_rate}_fr{cfg.freeze_ratio}"

        # load & wrap ResNet50 for 10 classes, tab_dim=0
        base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        model = ResNetWithTabular(base_model=base_model, tab_dim=0, num_classes=10)

        #  wrap  ImageFolder splits
        train_adapter = ImageOnlyDataset(train_ds)
        val_adapter   = ImageOnlyDataset(val_ds)

        # init FineTuneCNN exactly as before
        finetuner = FineTuneCNN(
            train_ds=train_adapter,
            valid_ds=val_adapter,
            base_model=model,
            batch_size=cfg.batch_size,
            freeze_ratio=cfg.freeze_ratio,
            test_ds=None     # no test set in this sweep
        )

        #  train & validate
        finetuner.run_training(
            num_epochs=cfg.epochs,
            learning_rate=cfg.learning_rate,
            weight_decay_val=cfg.l2_reg
        )

# launch the sweep agent
wandb.agent(sweep_id, function=main, count=20)

wandb: Agent Starting Run: pzjvo1u9 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	freeze_ratio: 0.9
wandb: 	l2_reg: 0.0005
wandb: 	learning_rate: 0.001
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 195MB/s] 


Epoch 1 — Train Acc: 51.63%
Epoch 1 — Val   Acc: 57.46%
Epoch 2 — Train Acc: 57.27%
Epoch 2 — Val   Acc: 62.20%
Epoch 3 — Train Acc: 59.54%
Epoch 3 — Val   Acc: 61.10%
Epoch 4 — Train Acc: 61.29%
Epoch 4 — Val   Acc: 62.66%
Epoch 5 — Train Acc: 62.94%
Epoch 5 — Val   Acc: 63.31%


epoch,▁▃▅▆█
train_acc,▁▄▆▇█
validation_accuracy,▁▇▅▇█
validation_loss,▄▁▆▇█
epoch,5
train_acc,62.94364
validation_accuracy,63.30571
validation_loss,1.27204


wandb: Agent Starting Run: juzejqtv with config:
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	freeze_ratio: 0.9
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0001


...

Epoch 1 — Train Acc: 58.96%
Epoch 1 — Val   Acc: 68.19%
Epoch 2 — Train Acc: 71.39%
Epoch 2 — Val   Acc: 68.88%
Epoch 3 — Train Acc: 78.86%
Epoch 3 — Val   Acc: 69.34%
Epoch 4 — Train Acc: 85.76%
Epoch 4 — Val   Acc: 71.64%
Epoch 5 — Train Acc: 91.02%
Epoch 5 — Val   Acc: 69.75%
Epoch 6 — Train Acc: 94.83%
Epoch 6 — Val   Acc: 71.13%
Epoch 7 — Train Acc: 95.97%
Epoch 7 — Val   Acc: 70.17%
Epoch 8 — Train Acc: 96.59%
Epoch 8 — Val   Acc: 70.76%
Epoch 9 — Train Acc: 97.25%
Epoch 9 — Val   Acc: 68.23%
Epoch 10 — Train Acc: 97.43%
Epoch 10 — Val   Acc: 70.99%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▅▆▇█████
validation_accuracy,▁▂▃█▄▇▅▆▁▇
validation_loss,▂▁▃▅▃▁▆█▆▅
epoch,10
train_acc,97.43275
validation_accuracy,70.99448
validation_loss,1.21297


wandb: Agent Starting Run: 38gtxm3g with config:
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	freeze_ratio: 0.2
wandb: 	l2_reg: 0
wandb: 	learning_rate: 0.0001


Epoch 1 — Train Acc: 59.11%
Epoch 1 — Val   Acc: 66.48%
Epoch 2 — Train Acc: 70.47%
Epoch 2 — Val   Acc: 68.37%
Epoch 3 — Train Acc: 78.75%
Epoch 3 — Val   Acc: 70.81%
Epoch 4 — Train Acc: 85.77%
Epoch 4 — Val   Acc: 67.73%
Epoch 5 — Train Acc: 91.52%
Epoch 5 — Val   Acc: 69.34%
Epoch 6 — Train Acc: 94.43%
Epoch 6 — Val   Acc: 68.05%
Epoch 7 — Train Acc: 95.99%
Epoch 7 — Val   Acc: 68.23%
Epoch 8 — Train Acc: 96.55%
Epoch 8 — Val   Acc: 69.01%
Epoch 9 — Train Acc: 97.22%
Epoch 9 — Val   Acc: 71.04%
Epoch 10 — Train Acc: 97.48%
Epoch 10 — Val   Acc: 71.36%


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▅▆▇▇████
validation_accuracy,▁▄▇▃▅▃▄▅██
validation_loss,▁▁▃▄▃▄█▇▅▆
epoch,10
train_acc,97.47878
validation_accuracy,71.3628
validation_loss,1.3452
